In [1]:
import os
import pandas as pd
from scipy.special import softmax
import numpy as np
import json

In [2]:
res_files = os.listdir('./ensemble_results/')

In [3]:
transformToModelToFile = {}
for f in res_files:
    x = f[:-4].split('_')
    m = x[0]
    t = "_".join(x[1:])
    if(t not in transformToModelToFile):
        transformToModelToFile[t] = {}
    transformToModelToFile[t][m] = os.path.join('./ensemble_results/', f)

In [14]:
def get_score_dict(modelToFiles, weights):
    dfDict = {}
    dfDict['resnet152'] = pd.read_csv(modelToFiles['resnet152']).set_index('Unnamed: 0')
    dfDict['vgg16'] = pd.read_csv(modelToFiles['vgg16']).set_index('Unnamed: 0')
    dfDict['alexnet'] = pd.read_csv(modelToFiles['alexnet']).set_index('Unnamed: 0')
    ensembled = {}
    for model in ['resnet152', 'vgg16', 'alexnet']:
        df = dfDict[model]
        images = list(df.index)
        npArray = df.to_numpy()
        w = weights[model]
        for i, img in enumerate(images):
            n = npArray[i]
            ns = w*softmax(n)/sum(weights.values())
            if(img not in ensembled):
                ensembled[img] = np.zeros((1000))
            ensembled[img] += ns
    return ensembled

def vote(x):
    maxIdx = np.argmax(x)
    v = np.zeros((x.shape))
    v[maxIdx] = 1.0
    return v
def get_vote_dict(modelToFiles, weights):
    dfDict = {}
    dfDict['resnet152'] = pd.read_csv(modelToFiles['resnet152']).set_index('Unnamed: 0')
    dfDict['vgg16'] = pd.read_csv(modelToFiles['vgg16']).set_index('Unnamed: 0')
    dfDict['alexnet'] = pd.read_csv(modelToFiles['alexnet']).set_index('Unnamed: 0')
    ensembled = {}
    for model in ['resnet152', 'vgg16', 'alexnet']:
        df = dfDict[model]
        images = list(df.index)
        npArray = df.to_numpy()
        w = weights[model]
        for i, img in enumerate(images):
            n = npArray[i]
            ns = w*vote(n)
            if(img not in ensembled):
                ensembled[img] = np.zeros((1000))
            ensembled[img] += ns
    return ensembled

def get_maxProb_dict(modelToFiles):
    dfDict = {}
    dfDict['resnet152'] = pd.read_csv(modelToFiles['resnet152']).set_index('Unnamed: 0')
    dfDict['vgg16'] = pd.read_csv(modelToFiles['vgg16']).set_index('Unnamed: 0')
    dfDict['alexnet'] = pd.read_csv(modelToFiles['alexnet']).set_index('Unnamed: 0')
    ensembled = {}
    for model in ['resnet152', 'vgg16', 'alexnet']:
        df = dfDict[model]
        images = list(df.index)
        npArray = df.to_numpy()
        for i, img in enumerate(images):
            n = npArray[i]
            ns = vote(n)
            if(img not in ensembled):
                ensembled[img] = np.zeros((1000))
            for c in range(1000):
                ensembled[img][c] = max(ensembled[img][c], ns[c])
    return ensembled

In [5]:
def get_img_name(idx):
    idx = str(idx)
    return 'ILSVRC2012_val_' + idx.zfill(8)

def compute_accuracy(ensembled, annotations, topk = [1]):
    res = [0 for _ in topk]
    for imgIdx in x:
        imgName = get_img_name(imgIdx)
        correctLabel = int(annotations[imgName])
        preds = np.argsort(ensembled[imgIdx])[::-1]
        for i, k in enumerate(topk):
            if(correctLabel in preds[:k]):
                res[i] += 1
    return [r*100.0/len(ensembled) for r in res]

In [7]:
weights = {'resnet152': 1.2, 
            'vgg16': 1.1,
            'alexnet': 1.0}
annotations = json.load(open('./imagnet_classes.json'))
x = get_vote_dict(transformToModelToFile['blur3'], weights)
print(compute_accuracy(x, annotations, topk = [1]))

x = get_maxProb_dict(transformToModelToFile['blur3'])
print(compute_accuracy(x, annotations, topk = [1, 5]))

[77.3]
[66.3, 83.7]


In [8]:
transformToModelToFile.keys()

dict_keys(['rotate5', 'blur3', 'shear_0.5_0', 'AGthreshold', 'cb_2_50', 'shuffle_quad', 'bilateralFilter_9_150_150', 'blur5', 'randRotate_1_45', 'rotate15', 'horz_shift_0.25', 'bilateralFilter_5_75_75', 'shear_0_0.5', 'vert_shift_0.5', 'randRotate_10_20', 'bilateralFilter_9_75_75', 'Original', 'vert_shift_0.25', 'rotate45', 'cb_0.5_m50', 'randRotate_1_20', 'randRotate_1_10', 'cb_2_m50', 'shuffle_quad_rand', 'horz_shift_0.5', 'invert', 'cb_0.5_50', 'cb_2_0'])

In [15]:
weights = {'resnet152': 79.3, 
            'vgg16': 71.5,
            'alexnet': 56.5}
annotations = json.load(open('./imagnet_classes.json'))
results = {}
for transform in transformToModelToFile:
    ensembled_scores = get_score_dict(transformToModelToFile[transform], weights)
    r = compute_accuracy(ensembled_scores, annotations, topk = [1, 5])
    results[transform] = {"Top 1": r[0], "Top 5": r[1]}

In [16]:
pd.DataFrame.from_dict(results).transpose()

,Top 1,Top 5
rotate5,79.1,93.5
blur3,77.1,92.8
shear_0.5_0,68.8,86.3
AGthreshold,7.2,13.6
cb_2_50,65.1,84.2
shuffle_quad,57.0,76.4
bilateralFilter_9_150_150,71.8,89.7
blur5,77.1,91.9
randRotate_1_45,75.9,89.7
rotate15,77.4,91.5


In [21]:
weights = {'resnet152': 1.2, 
            'vgg16': 1.1,
            'alexnet': 1.0}
annotations = json.load(open('./imagnet_classes.json'))
results = {}
for transform in transformToModelToFile:
    ensembled_scores = get_vote_dict(transformToModelToFile[transform], weights)
    r = compute_accuracy(ensembled_scores, annotations, topk = [1])
    results[transform] = {"Top 1": r[0], "Top 5": 0}

In [22]:
pd.DataFrame.from_dict(results).transpose()

,Top 1,Top 5
rotate5,78.5,0.0
blur3,77.3,0.0
shear_0.5_0,66.4,0.0
AGthreshold,7.3,0.0
cb_2_50,63.9,0.0
shuffle_quad,55.2,0.0
bilateralFilter_9_150_150,71.4,0.0
blur5,75.3,0.0
randRotate_1_45,73.3,0.0
rotate15,75.3,0.0


In [23]:
annotations = json.load(open('./imagnet_classes.json'))
results = {}
for transform in transformToModelToFile:
    ensembled_scores = get_maxProb_dict(transformToModelToFile[transform])
    r = compute_accuracy(ensembled_scores, annotations, topk = [1, 5])
    results[transform] = {"Top 1": r[0], "Top 5": r[1]}

In [24]:
pd.DataFrame.from_dict(results).transpose()

,Top 1,Top 5
rotate5,66.6,83.4
blur3,66.3,83.7
shear_0.5_0,50.4,74.5
AGthreshold,4.3,10.2
cb_2_50,47.2,71.6
shuffle_quad,35.2,62.8
bilateralFilter_9_150_150,57.6,79.1
blur5,64.6,82.3
randRotate_1_45,57.3,80.2
rotate15,62.9,82.6
